In [ ]:
# 必要なライブラリをインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import logging
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sqlalchemy import create_engine, text
import torch.nn.functional as F
import japanize_matplotlib

# 自作モジュールをインポート
from _dataset import HorguesDataset
from _models import HorguesModel
from horgues3.losses import HorguesLoss
from horgues3.odds import get_odds_dataframes
from horgues3.haraimodoshi import get_haraimodoshi_dataframes
from horgues3.database import create_database_engine

In [ ]:
# ログ設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [ ]:
# デバイス設定
device = torch.device('cuda:1')
logger.info(f"使用デバイス: {device}")

In [ ]:
# 学習パラメータ設定
BATCH_SIZE = 128

In [ ]:
checkpoint = torch.load('outputs/best_model.pth', map_location='cpu')

In [ ]:
# 学習期間・検証期間設定
START_DATE = '20230101'
END_DATE = '20231231'

logger.info(f"期間: {START_DATE} - {END_DATE}")

In [ ]:
# データセットの作成
logger.info("データセットを作成中...")
dataset = HorguesDataset(
    start_date=START_DATE,
    end_date=END_DATE,
    preprocessing_params=checkpoint['preprocessing_params'],
    cache_dir='cache/test',
    use_cache=True
)

logger.info(f"データサイズ: {len(dataset)}")

In [ ]:
loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

In [ ]:
# モデル設定の取得
model_config = checkpoint['model_config']
logger.info(f"数値特徴量数: {len(model_config['numerical_features'])}")
logger.info(f"カテゴリ特徴量数: {len(model_config['categorical_features'])}")
logger.info(f"時系列データ: {model_config['sequence_names']}")

In [ ]:
# モデルの作成
model = HorguesModel(
    sequence_names=model_config['sequence_names'],
    feature_aliases=model_config['feature_aliases'],
    numerical_features=model_config['numerical_features'],
    categorical_features=model_config['categorical_features'],
).to(device)

In [ ]:
model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
race_ids = []
masks = []
targets = []
scores = []

model.eval()
with torch.no_grad():
    pbar = tqdm(loader, desc='Predict')
    for batch in pbar:
        race_ids.extend(batch['race_id'])
        masks.append(batch['mask'])
        targets.append(batch['target'])

        x_num = {k: v.to(device) for k, v in batch['x_num'].items()}
        x_cat = {k: v.to(device) for k, v in batch['x_cat'].items()}
        sequence_data = {}
        for seq_name, seq_data in batch['sequence_data'].items():
            sequence_data[seq_name] = {
                'x_num': {k: v.to(device) for k, v in seq_data['x_num'].items()},
                'x_cat': {k: v.to(device) for k, v in seq_data['x_cat'].items()},
                'mask': seq_data['mask'].to(device)
            }
        mask = batch['mask'].to(device)
        target = batch['target'].to(device)

        scores.append(model(x_num, x_cat, sequence_data, mask).cpu())

    masks = torch.cat(masks, dim=0)
    targets = torch.cat(targets, dim=0)
    scores = torch.cat(scores, dim=0)

In [ ]:
odds = get_odds_dataframes(START_DATE, END_DATE)
haraimodoshi = get_haraimodoshi_dataframes(START_DATE, END_DATE)

In [ ]:
def convert_to_series(tensor, name):
    tmp = pd.DataFrame(tensor, index=race_ids, columns=np.arange(1, 19))
    tmp = tmp.reset_index(names='race_id').melt(id_vars='race_id', var_name='uma_number')
    tmp = tmp.set_index(['race_id', 'uma_number']).value.rename(name)
    return tmp

In [ ]:
df = pd.DataFrame(
    {
        'score_0': convert_to_series(F.sigmoid(scores[:, :, 0]), 'score_0'),
        'score_1': convert_to_series(F.sigmoid(scores[:, :, 1]), 'score_1'),
        'score_2': convert_to_series(F.sigmoid(scores[:, :, 2]), 'score_2'),
        'target_0': convert_to_series(targets[:, :, 0], 'target_0'),
        'target_1': convert_to_series(targets[:, :, 1], 'target_1'),
        'target_2': convert_to_series(targets[:, :, 2], 'target_2'),
        'mask': convert_to_series(masks, 'mask'),
    }
)
df = df[df['mask']].sort_index()

In [ ]:
df['score_ranking_0'] = df.groupby('race_id')['score_0'].rank(method='first', ascending=False)
df['score_ranking_1'] = df.groupby('race_id')['score_1'].rank(method='first', ascending=False)
df['score_ranking_2'] = df.groupby('race_id')['score_2'].rank(method='first', ascending=False)

In [ ]:
# 全券種対応のシミュレーション関数（新評価方法対応）
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import product, combinations, permutations
import multiprocessing as mp

def simulate_betting_all_types_new_eval(df, odds_data, haraimodoshi_data, bet_type, min_odds_range, 
                                       score_0_threshold, score_1_threshold, score_2_threshold,
                                       score_ranking_0_threshold, score_ranking_1_threshold, score_ranking_2_threshold):
    """
    全券種対応ベッティングシミュレーション（新評価方法：調整済み回収率**的中回数）
    
    Parameters:
    -----------
    df : pd.DataFrame
        予測結果データ
    odds_data : dict
        オッズデータ
    haraimodoshi_data : dict  
        払戻データ
    bet_type : str
        券種 ('tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan')
    min_odds_range : list
        最低オッズの範囲
    score_0_threshold : float
        1着スコア閾値
    score_1_threshold : float
        2着スコア閾値
    score_2_threshold : float
        3着スコア閾値
    score_ranking_0_threshold : int
        1着ランキング閾値
    score_ranking_1_threshold : int
        2着ランキング閾値
    score_ranking_2_threshold : int
        3着ランキング閾値
    
    Returns:
    --------
    pd.DataFrame
        シミュレーション結果
    """
    results = []
    
    if bet_type not in odds_data or bet_type not in haraimodoshi_data:
        return pd.DataFrame()
    
    odds_df = odds_data[bet_type]
    haraimodoshi_df = haraimodoshi_data[bet_type]
    
    for min_odds in min_odds_range:
        total_cost = 0
        total_return = 0
        total_races = 0
        hit_races = 0  # 的中レース数
        race_results = []
        
        # レース毎にシミュレーション
        for race_id in df.index.get_level_values('race_id').unique():
            if race_id not in odds_df.index or race_id not in haraimodoshi_df.index:
                continue
                
            race_df = df.loc[race_id]
            
            # 券種別の購入候補選択
            purchase_combinations = []
            race_cost = 0
            
            if bet_type == 'tansho':
                # 単勝：1着候補のみ
                candidates = race_df[
                    (race_df['score_0'] >= score_0_threshold) & 
                    (race_df['score_ranking_0'] <= score_ranking_0_threshold)
                ].index.tolist()
                
                for horse in candidates:
                    horse_key = f'{horse:02d}'
                    if horse_key in odds_df.columns:
                        odds_val = odds_df.loc[race_id, horse_key]
                        if odds_val >= min_odds and odds_val > 0:
                            purchase_combinations.append((horse_key, odds_val))
                            race_cost += 100
                            
            elif bet_type == 'fukusho':
                # 複勝：1-3着候補
                candidates = race_df[
                    ((race_df['score_0'] >= score_0_threshold) & (race_df['score_ranking_0'] <= score_ranking_0_threshold)) |
                    ((race_df['score_1'] >= score_1_threshold) & (race_df['score_ranking_1'] <= score_ranking_1_threshold)) |
                    ((race_df['score_2'] >= score_2_threshold) & (race_df['score_ranking_2'] <= score_ranking_2_threshold))
                ].index.tolist()
                
                for horse in candidates:
                    horse_key = f'{horse:02d}'
                    if horse_key in odds_df.columns:
                        odds_val = odds_df.loc[race_id, horse_key]
                        if odds_val >= min_odds and odds_val > 0:
                            purchase_combinations.append((horse_key, odds_val))
                            race_cost += 100
                            
            elif bet_type in ['umaren', 'wide']:
                # 馬連・ワイド：1着候補×2着候補 + 2着候補×1着候補の組み合わせ
                first_candidates = race_df[
                    (race_df['score_0'] >= score_0_threshold) & 
                    (race_df['score_ranking_0'] <= score_ranking_0_threshold)
                ].index.tolist()
                
                second_candidates = race_df[
                    (race_df['score_1'] >= score_1_threshold) & 
                    (race_df['score_ranking_1'] <= score_ranking_1_threshold)
                ].index.tolist()
                
                all_candidates = list(set(first_candidates + second_candidates))
                
                for i, j in combinations(all_candidates, 2):
                    combination_key = f'{min(i,j):02d}-{max(i,j):02d}'
                    if combination_key in odds_df.columns:
                        odds_val = odds_df.loc[race_id, combination_key]
                        if odds_val >= min_odds and odds_val > 0:
                            purchase_combinations.append((combination_key, odds_val))
                            race_cost += 100
                            
            elif bet_type == 'umatan':
                # 馬単：1着候補→2着候補の順列
                first_candidates = race_df[
                    (race_df['score_0'] >= score_0_threshold) & 
                    (race_df['score_ranking_0'] <= score_ranking_0_threshold)
                ].index.tolist()
                
                second_candidates = race_df[
                    (race_df['score_1'] >= score_1_threshold) & 
                    (race_df['score_ranking_1'] <= score_ranking_1_threshold)
                ].index.tolist()
                
                for first in first_candidates:
                    for second in second_candidates:
                        if first != second:
                            combination_key = f'{first:02d}-{second:02d}'
                            if combination_key in odds_df.columns:
                                odds_val = odds_df.loc[race_id, combination_key]
                                if odds_val >= min_odds and odds_val > 0:
                                    purchase_combinations.append((combination_key, odds_val))
                                    race_cost += 100
                                    
            elif bet_type == 'sanrenpuku':
                # 三連複：1着、2着、3着候補の組み合わせ
                first_candidates = race_df[
                    (race_df['score_0'] >= score_0_threshold) & 
                    (race_df['score_ranking_0'] <= score_ranking_0_threshold)
                ].index.tolist()
                
                second_candidates = race_df[
                    (race_df['score_1'] >= score_1_threshold) & 
                    (race_df['score_ranking_1'] <= score_ranking_1_threshold)
                ].index.tolist()
                
                third_candidates = race_df[
                    (race_df['score_2'] >= score_2_threshold) & 
                    (race_df['score_ranking_2'] <= score_ranking_2_threshold)
                ].index.tolist()
                
                all_candidates = list(set(first_candidates + second_candidates + third_candidates))
                
                for i, j, k in combinations(all_candidates, 3):
                    sorted_horses = sorted([i, j, k])
                    combination_key = f'{sorted_horses[0]:02d}-{sorted_horses[1]:02d}-{sorted_horses[2]:02d}'
                    if combination_key in odds_df.columns:
                        odds_val = odds_df.loc[race_id, combination_key]
                        if odds_val >= min_odds and odds_val > 0:
                            purchase_combinations.append((combination_key, odds_val))
                            race_cost += 100
                            
            elif bet_type == 'sanrentan':
                # 三連単：1着→2着→3着の順列
                first_candidates = race_df[
                    (race_df['score_0'] >= score_0_threshold) & 
                    (race_df['score_ranking_0'] <= score_ranking_0_threshold)
                ].index.tolist()
                
                second_candidates = race_df[
                    (race_df['score_1'] >= score_1_threshold) & 
                    (race_df['score_ranking_1'] <= score_ranking_1_threshold)
                ].index.tolist()
                
                third_candidates = race_df[
                    (race_df['score_2'] >= score_2_threshold) & 
                    (race_df['score_ranking_2'] <= score_ranking_2_threshold)
                ].index.tolist()
                
                for first in first_candidates:
                    for second in second_candidates:
                        for third in third_candidates:
                            if len(set([first, second, third])) == 3:  # 全て異なる馬
                                combination_key = f'{first:02d}-{second:02d}-{third:02d}'
                                if combination_key in odds_df.columns:
                                    odds_val = odds_df.loc[race_id, combination_key]
                                    if odds_val >= min_odds and odds_val > 0:
                                        purchase_combinations.append((combination_key, odds_val))
                                        race_cost += 100
            
            if len(purchase_combinations) == 0:
                continue
                
            total_cost += race_cost
            total_races += 1
            
            # 払戻計算
            race_return = 0
            race_payouts = []
            race_hit = False  # このレースで的中したかどうか
            
            for combination_key, odds_val in purchase_combinations:
                if combination_key in haraimodoshi_df.columns:
                    payout = haraimodoshi_df.loc[race_id, combination_key]
                    if payout > 0:
                        race_return += payout
                        race_payouts.append(payout)
                        race_hit = True
            
            if race_hit:
                hit_races += 1
            
            total_return += race_return
            race_results.append({
                'race_id': race_id,
                'cost': race_cost,
                'return': race_return,
                'profit': race_return - race_cost,
                'combinations': len(purchase_combinations),
                'max_payout': max(race_payouts) if race_payouts else 0,
                'hit': race_hit
            })
        
        # 一発屋対策：最高払戻額を差し引く
        max_payout = max([r['max_payout'] for r in race_results]) if race_results else 0
        adjusted_return = total_return - max_payout
        adjusted_profit = adjusted_return - total_cost
        
        # 基本指標計算
        return_rate = total_return / total_cost if total_cost > 0 else 0
        adjusted_return_rate = adjusted_return / total_cost if total_cost > 0 else 0
        hit_rate = hit_races / total_races if total_races > 0 else 0
        
        # 新評価指標：調整済み回収率の的中回数乗
        # 的中回数が0の場合は評価値を0とする
        if hit_races > 0 and adjusted_return_rate > 0:
            power_evaluation = adjusted_return_rate ** hit_races
        else:
            power_evaluation = 0
        
        results.append({
            'bet_type': bet_type,
            'min_odds': min_odds,
            'total_races': total_races,
            'hit_races': hit_races,
            'hit_rate': hit_rate,
            'total_cost': total_cost,
            'total_return': total_return,
            'adjusted_return': adjusted_return,
            'total_profit': total_return - total_cost,
            'adjusted_profit': adjusted_profit,
            'max_payout': max_payout,
            'return_rate': return_rate,
            'adjusted_return_rate': adjusted_return_rate,
            'power_evaluation': power_evaluation,  # 新評価指標
            'score_0_threshold': score_0_threshold,
            'score_1_threshold': score_1_threshold,
            'score_2_threshold': score_2_threshold,
            'score_ranking_0_threshold': score_ranking_0_threshold,
            'score_ranking_1_threshold': score_ranking_1_threshold,
            'score_ranking_2_threshold': score_ranking_2_threshold
        })
    
    return pd.DataFrame(results)


In [ ]:
def worker_function_new_eval(args):
    """新評価方法対応ワーカー関数（並列処理用）"""
    (df, odds_data, haraimodoshi_data, bet_type, min_odds_range, 
     score_0_threshold, score_1_threshold, score_2_threshold,
     score_ranking_0_threshold, score_ranking_1_threshold, score_ranking_2_threshold) = args
    
    try:
        result = simulate_betting_all_types_new_eval(
            df, odds_data, haraimodoshi_data, bet_type, min_odds_range,
            score_0_threshold, score_1_threshold, score_2_threshold,
            score_ranking_0_threshold, score_ranking_1_threshold, score_ranking_2_threshold
        )
        
        if len(result) > 0:
            # 新評価指標（power_evaluation）で最大値を選択
            max_idx = result['power_evaluation'].idxmax()
            best_result = result.loc[max_idx].copy()
            best_result['full_results'] = result
            return best_result
        else:
            return None
    except Exception as e:
        print(f"Error in worker for {bet_type}: {e}")
        return None


In [ ]:
# 全券種シミュレーション実行（新評価方法）
print("全券種対応シミュレーション開始（新評価方法：調整済み回収率**的中回数）...")

# 券種リスト
bet_types = ['tansho', 'fukusho', 'umaren', 'wide', 'umatan', 'sanrenpuku', 'sanrentan']

# 閾値設定（簡略化）
score_0_thresholds = [0.1, 0.2, 0.3]
score_1_thresholds = [0.1, 0.2, 0.3]
score_2_thresholds = [0.1, 0.2, 0.3]
score_ranking_0_thresholds = [1, 2, 3]
score_ranking_1_thresholds = [18]
score_ranking_2_thresholds = [18]

# オッズ範囲
min_odds_range = np.arange(1.0, 21.0, 5.0)

# 全組み合わせ生成
all_combinations = []
for bet_type in bet_types:
    for s0, s1, s2, sr0, sr1, sr2 in product(
        score_0_thresholds, score_1_thresholds, score_2_thresholds,
        score_ranking_0_thresholds, score_ranking_1_thresholds, score_ranking_2_thresholds
    ):
        all_combinations.append((
            df, odds, haraimodoshi, bet_type, min_odds_range,
            s0, s1, s2, sr0, sr1, sr2
        ))

print(f"全組み合わせ数: {len(all_combinations)}")

# 並列実行
all_betting_results_new = []
n_processes = min(mp.cpu_count(), 8)

with ProcessPoolExecutor(max_workers=n_processes) as executor:
    future_to_params = {
        executor.submit(worker_function_new_eval, args): args[3]  # bet_type
        for args in all_combinations
    }
    
    with tqdm(total=len(all_combinations), desc='新評価方法最適化進行中') as pbar:
        for future in as_completed(future_to_params):
            result = future.result()
            if result is not None:
                all_betting_results_new.append(result)
            pbar.update(1)

print(f"新評価方法最適化完了: {len(all_betting_results_new)}個の有効な結果")



In [ ]:
# 券種別結果分析（新評価方法）
if all_betting_results_new:
    all_results_new_df = pd.DataFrame(all_betting_results_new)
    
    # 券種別の最優秀結果（新評価指標順）
    print("\n=== 券種別最優秀結果（新評価方法：調整済み回収率**的中回数） ===")
    best_by_type_new = all_results_new_df.loc[all_results_new_df.groupby('bet_type')['power_evaluation'].idxmax()]
    best_summary_new = best_by_type_new[['bet_type', 'power_evaluation', 'adjusted_return_rate', 'hit_rate', 'hit_races', 
                                        'total_races', 'adjusted_profit', 'min_odds']].copy()
    
    # 数値フォーマット調整
    best_summary_new['power_evaluation'] = best_summary_new['power_evaluation'].apply(lambda x: f'{x:.6f}')
    best_summary_new['adjusted_return_rate'] = best_summary_new['adjusted_return_rate'].apply(lambda x: f'{x:.3f}')
    best_summary_new['hit_rate'] = best_summary_new['hit_rate'].apply(lambda x: f'{x:.3f}')
    best_summary_new['adjusted_profit'] = best_summary_new['adjusted_profit'].apply(lambda x: f'{x:,.0f}')
    
    print(best_summary_new.to_string(index=False))
    
    # 詳細結果グラフ作成
    plt.figure(figsize=(20, 15))
    
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink']
    
    # 新評価指標のグラフ
    for i, (bet_type, color) in enumerate(zip(bet_types, colors)):
        type_results = all_results_new_df[all_results_new_df['bet_type'] == bet_type]
        if len(type_results) > 0:
            best_result = type_results.loc[type_results['power_evaluation'].idxmax()]
            full_results = best_result['full_results']
            
            plt.subplot(3, 3, i + 1)
            plt.plot(full_results['min_odds'], full_results['power_evaluation'], 
                    color=color, linewidth=2, marker='o', markersize=4)
            plt.xlabel('最低オッズ')
            plt.ylabel('評価値 (回収率**的中回数)')
            plt.title(f'{bet_type}\n最大評価値: {best_result["power_evaluation"]:.6f}')
            plt.grid(True, alpha=0.3)
            plt.yscale('log')  # 対数スケール
    
    # 調整済み利益のグラフ
    plt.subplot(3, 3, 8)
    for bet_type, color in zip(bet_types, colors):
        type_results = all_results_new_df[all_results_new_df['bet_type'] == bet_type]
        if len(type_results) > 0:
            best_result = type_results.loc[type_results['power_evaluation'].idxmax()]
            full_results = best_result['full_results']
            plt.plot(full_results['min_odds'], full_results['adjusted_profit'], 
                    color=color, linewidth=2, marker='o', markersize=3, label=bet_type)
    plt.xlabel('最低オッズ')
    plt.ylabel('調整済み利益 (円)')
    plt.title('券種別調整済み利益比較')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 回収率vs的中率のグラフ
    plt.subplot(3, 3, 9)
    for bet_type, color in zip(bet_types, colors):
        type_results = all_results_new_df[all_results_new_df['bet_type'] == bet_type]
        if len(type_results) > 0:
            best_result = type_results.loc[type_results['power_evaluation'].idxmax()]
            plt.scatter(best_result['hit_rate'], best_result['adjusted_return_rate'], 
                       color=color, s=100, label=bet_type, alpha=0.7)
    plt.xlabel('的中率')
    plt.ylabel('調整済み回収率')
    plt.title('的中率 vs 回収率')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='損益分岐点')
    
    plt.tight_layout()
    plt.show()
    
    # 券種別パフォーマンス比較（新評価指標）
    plt.figure(figsize=(15, 10))
    
    # 新評価指標の棒グラフ
    plt.subplot(2, 2, 1)
    bars1 = plt.bar(best_by_type_new['bet_type'], 
                   best_by_type_new['power_evaluation'].astype(float), 
                   color=colors[:len(best_by_type_new)])
    plt.xlabel('券種')
    plt.ylabel('評価値 (回収率**的中回数)')
    plt.title('券種別最大評価値')
    plt.xticks(rotation=45)
    plt.yscale('log')
    
    # 調整済み利益の棒グラフ
    plt.subplot(2, 2, 2)
    bars2 = plt.bar(best_by_type_new['bet_type'], 
                   best_by_type_new['adjusted_profit'].astype(str).str.replace(',', '').astype(float), 
                   color=colors[:len(best_by_type_new)])
    plt.xlabel('券種')
    plt.ylabel('調整済み利益 (円)')
    plt.title('券種別最大調整済み利益')
    plt.xticks(rotation=45)
    
    # 的中率の棒グラフ
    plt.subplot(2, 2, 3)
    bars3 = plt.bar(best_by_type_new['bet_type'], 
                   best_by_type_new['hit_rate'].astype(float), 
                   color=colors[:len(best_by_type_new)])
    plt.xlabel('券種')
    plt.ylabel('的中率')
    plt.title('券種別的中率')
    plt.xticks(rotation=45)
    
    # 調整済み回収率の棒グラフ
    plt.subplot(2, 2, 4)
    bars4 = plt.bar(best_by_type_new['bet_type'], 
                   best_by_type_new['adjusted_return_rate'].astype(float), 
                   color=colors[:len(best_by_type_new)])
    plt.xlabel('券種')
    plt.ylabel('調整済み回収率')
    plt.title('券種別調整済み回収率')
    plt.xticks(rotation=45)
    plt.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='損益分岐点')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 総合最優秀結果（新評価指標）
    overall_best_new = all_results_new_df.loc[all_results_new_df['power_evaluation'].idxmax()]
    print(f"\n=== 全券種中最優秀結果（新評価方法） ===")
    print(f"券種: {overall_best_new['bet_type']}")
    print(f"評価値 (回収率**的中回数): {overall_best_new['power_evaluation']:.6f}")
    print(f"調整済み回収率: {overall_best_new['adjusted_return_rate']:.3f}")
    print(f"的中率: {overall_best_new['hit_rate']:.3f}")
    print(f"的中回数: {int(overall_best_new['hit_races'])}回")
    print(f"対象レース数: {int(overall_best_new['total_races'])}レース")
    print(f"調整済み利益: {overall_best_new['adjusted_profit']:,.0f}円")
    print(f"最適最低オッズ: {overall_best_new['min_odds']:.1f}倍")
    print(f"最適閾値設定:")
    print(f"  score_0閾値: {overall_best_new['score_0_threshold']:.3f}")
    print(f"  score_1閾値: {overall_best_new['score_1_threshold']:.3f}")
    print(f"  score_2閾値: {overall_best_new['score_2_threshold']:.3f}")
    print(f"  score_ranking_0閾値: {int(overall_best_new['score_ranking_0_threshold'])}")
    print(f"  score_ranking_1閾値: {int(overall_best_new['score_ranking_1_threshold'])}")
    print(f"  score_ranking_2閾値: {int(overall_best_new['score_ranking_2_threshold'])}")
    
    # 上位10位の詳細表示
    print(f"\n=== 上位10位詳細（新評価方法） ===")
    top_10_new = all_results_new_df.nlargest(10, 'power_evaluation')[
        ['bet_type', 'power_evaluation', 'adjusted_return_rate', 'hit_rate', 'hit_races', 'adjusted_profit', 'min_odds']
    ].copy()
    print(top_10_new.to_string(index=False))

else:
    print("有効な結果が得られませんでした。")
